<a href="https://colab.research.google.com/github/sljssims/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import your dependencies
import os
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [829 kB]
Hit:13 http://ppa.laun

In [3]:
# Import SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine-Analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Extract the dataset you used in Deliverable 1
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [ ]:
# # Look at the dataframe
# from pyspark.sql.functions import to_date
# # Read in the Review dataset as a DataFrame
# df.show(5)

In [6]:
#  Recreate the vine_table dataframe
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "vine", "verified_purchase", "total_votes"])
vine_df.show()

+--------------+-----------+-------------+----+-----------------+-----------+
|     review_id|star_rating|helpful_votes|vine|verified_purchase|total_votes|
+--------------+-----------+-------------+----+-----------------+-----------+
| REAKC26P07MDN|          5|            0|   N|                Y|          0|
|R3NU7OMZ4HQIEG|          2|            0|   N|                Y|          1|
|R14QJW3XF8QO1P|          5|            0|   N|                Y|          0|
|R2HB7AX0394ZGY|          5|            0|   N|                Y|          0|
| RGKMPDQGSAHR3|          5|            0|   N|                Y|          0|
|R1DJCVPQGCV66E|          5|            0|   N|                Y|          0|
|R3V52EAWLPBFQG|          3|            0|   N|                Y|          0|
|R3DKO8J1J28QBI|          2|            0|   N|                Y|          0|
| R764DBXGRNECG|          5|            1|   N|                N|          1|
| RW1853GAT0Z9F|          5|            0|   N|                Y

In [7]:
# Filter the data for total_votes > 20
vote_count = vine_df.filter(vine_df["total_votes"] >= 20)



In [8]:
# Create a new DataFrame to retrieve all rows where # of helpful_votes divided by total_votes is equal to or greater than 50%.
new_vote_table = vote_count.filter(vote_count["helpful_votes"]/vote_count["total_votes"]>=0.5)
new_vote_table.show()

+--------------+-----------+-------------+----+-----------------+-----------+
|     review_id|star_rating|helpful_votes|vine|verified_purchase|total_votes|
+--------------+-----------+-------------+----+-----------------+-----------+
|R21KC552Y6HL8X|          1|           27|   N|                Y|         31|
| RX9WC9FTIR1XR|          5|           25|   N|                Y|         25|
| RGDCOU1KBHMNG|          3|           29|   N|                Y|         31|
| RVTYWID2TPMMY|          2|           35|   N|                Y|         42|
|R2CMPZ5VESGRLY|          4|           27|   N|                Y|         28|
|R3VQPJZ54B55BA|          5|           62|   N|                N|         64|
|R24QM6D7FEDZ5M|          2|           36|   N|                Y|         43|
|R3A1VQ3SQDXEJ3|          5|           20|   N|                Y|         20|
|  R39GSNW76GYF|          1|           20|   N|                Y|         23|
| RPJLR6MFDPXXE|          5|           35|   N|                Y

In [9]:
# Filter the table created in Step 2, and create a new table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'
vine_votes = new_vote_table.filter(new_vote_table["vine"] == "Y").count()

In [10]:
# Create a new table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'
non_vine_votes = new_vote_table.filter(new_vote_table["vine"] == "N").count()

In [11]:
# Filter table for five star reviews
five_star = new_vote_table.filter(new_vote_table["star_rating"]== 5)
five_star.show()

+--------------+-----------+-------------+----+-----------------+-----------+
|     review_id|star_rating|helpful_votes|vine|verified_purchase|total_votes|
+--------------+-----------+-------------+----+-----------------+-----------+
| RX9WC9FTIR1XR|          5|           25|   N|                Y|         25|
|R3VQPJZ54B55BA|          5|           62|   N|                N|         64|
|R3A1VQ3SQDXEJ3|          5|           20|   N|                Y|         20|
| RPJLR6MFDPXXE|          5|           35|   N|                Y|         36|
|R34X0ZGVB3YGYO|          5|           47|   N|                Y|         47|
| RM6ZH5J3XDNHK|          5|           79|   N|                Y|         87|
|R2E363K75EMW4K|          5|           66|   N|                Y|         69|
| RLMZOQL11VHBX|          5|           23|   N|                Y|         24|
|R3C6CQQRRA8SYP|          5|           52|   N|                Y|         58|
|R176V3AYS3R4HG|          5|           26|   N|                N

In [12]:
# Determine the total number of reviews
new_vote_table.count()

38010

In [13]:
# Determine the total number of 5-star reviews
five_star.count()

20677

In [14]:
# Percentage of five star reviews
five_star.count() / new_vote_table.count()

0.5439884240989213

In [15]:
# Determine number of 5-star reviews for the Vine members
five_star.filter(five_star["verified_purchase"] == "Y").count()

14950

In [16]:
# Determine number of 5-star reviews for the non-Vine members
five_star.filter(five_star["verified_purchase"] == "N").count()

5727

In [17]:
# Determine the percentage of 5-star reviews for the paid reviews.
five_star.filter(five_star["verified_purchase"] == "Y").count()/new_vote_table.filter(new_vote_table["verified_purchase"]== "Y").count()

0.5740726518700561

In [18]:
# Determine the percentage of 5-star reviews for un-paid reviews.
five_star.filter(five_star["verified_purchase"] == "N").count()/new_vote_table.filter(new_vote_table["verified_purchase"]== "N").count()

0.47852606951871657